In [1]:
import numpy as np
import functools as ft
import networkx as nx

In [2]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def zero(d):
    return np.zeros((d,))


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)


def similarity(A, B):
    return np.dot(A, B) / len(A)


sim = cosine_similarity

In [3]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def getVector(self, label):
        return self.vectors[label]

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: sim(V, x[1]))

In [4]:
def initVertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [5]:
def initNodeMem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", graph.nodes[n]["mem"])

In [6]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if sim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if sim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.addVector(f"mem{n}_{i}", mem)

In [7]:
def initGraph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.addVector("g", G)
    return G

In [8]:
def checkEdge(G, A, B, threshold):
    return sim(B, bind([G, A])) > threshold

In [9]:
def nodeMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = nodeMemoryReconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [10]:
def graphMemoryReconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graphMemoryReconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [11]:
# visited = dict()
visited = {}


def shortestPath(G, A, B, xs, distance, threshold, memory):
    label, value = memory.cleanup(A)

    if np.array_equiv(A, B):
        return 0, label

    if label in visited and visited[label][0] <= distance:
        return visited[label][1], visited[label][2]

    visited[label] = [distance, 99, "memnill"]

    neighbours = list(filter(lambda x: checkEdge(G, A, x, threshold), xs))

    if len(neighbours) == 0:
        return 99, "memnill"

    dis, lab = min(
        list(
            map(
                lambda a: shortestPath(G, a, B, xs, distance + 1, threshold, memory),
                neighbours,
            )
        ),
        key=lambda x: x[0],
    )
    visited[label][1] = dis + 1
    visited[label][2] = label + lab
    return dis + 1, label + lab

---


# Tests


In [12]:
def testCheckEdge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = checkEdge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [13]:
def testNodeMemoryReconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = nodeMemoryReconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(sim(graph.nodes[0]["mem"], memsi[0])),
        )

---


# Main


In [14]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [15]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    initVertices(graph, DIMENSIONS)
    initNodeMem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # testNodeMemoryReconstruction(G, graph, 1)
    # testCheckEdge(G, graph, THRESHOLD)


# main()

In [16]:
# graph = nx.gnm_random_graph(NODES, EDGES)
# memory = ItemMemory()

# initVertices(graph, DIMENSIONS)
# initNodeMem(graph, memory)
# G = initGraph(graph, memory)
# print("G =>", G)

In [17]:
# testNodeMemoryReconstruction(G, graph, 15)

In [18]:
# sim(graph.nodes[0]["mem"], graph.nodes[0]["mem"])

---


In [23]:
mem = ItemMemory()

g = nx.gnm_random_graph(NODES, 45)
initVertices(g, DIMENSIONS)
initNodeMem(g, mem)
G = initGraph(g, mem)

for n in g.nodes():
    print(g.edges(n))

[(0, 10), (0, 27), (0, 18), (0, 29), (0, 24)]
[(1, 9), (1, 21)]
[(2, 16), (2, 7), (2, 4)]
[(3, 12)]
[(4, 27), (4, 2), (4, 15)]
[(5, 23), (5, 6), (5, 21)]
[(6, 21), (6, 5), (6, 18), (6, 23), (6, 7)]
[(7, 29), (7, 2), (7, 6)]
[(8, 19)]
[(9, 17), (9, 1)]
[(10, 0), (10, 28), (10, 29), (10, 14)]
[(11, 13)]
[(12, 15), (12, 17), (12, 3)]
[(13, 22), (13, 15), (13, 11), (13, 21), (13, 25)]
[(14, 10)]
[(15, 13), (15, 12), (15, 21), (15, 4)]
[(16, 20), (16, 17), (16, 2)]
[(17, 9), (17, 12), (17, 16), (17, 19), (17, 29)]
[(18, 20), (18, 6), (18, 0)]
[(19, 8), (19, 17)]
[(20, 16), (20, 18)]
[(21, 6), (21, 1), (21, 13), (21, 15), (21, 5)]
[(22, 13), (22, 28), (22, 29)]
[(23, 5), (23, 25), (23, 26), (23, 6)]
[(24, 0)]
[(25, 23), (25, 13)]
[(26, 23), (26, 29)]
[(27, 0), (27, 4)]
[(28, 29), (28, 10), (28, 22)]
[(29, 28), (29, 7), (29, 10), (29, 17), (29, 0), (29, 22), (29, 26)]


In [ ]:
hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
origin = 0

In [31]:
for n in g.nodes():
    print(
        f"{n:02} =>",
        "T" if mem.cleanup(hdvs[n])[0] == f"mem{n}" else "F",
        mem.cleanup(hdvs[n])[0],
    )

00 => mem24 False
01 => mem9 False
02 => mem7 False
03 => mem12 False
04 => mem27 False
05 => mem23 False
06 => mem5 False
07 => mem2 False
08 => mem19 False
09 => mem1 False
10 => mem14 False
11 => mem13 False
12 => mem3 False
13 => mem11 False
14 => mem10 False
15 => mem12 False
16 => mem20 False
17 => mem19 False
18 => mem20 False
19 => mem8 False
20 => mem16 False
21 => mem1 False
22 => mem28 False
23 => mem25 False
24 => mem0 False
25 => mem23 False
26 => mem23 False
27 => mem4 False
28 => mem22 False
29 => mem26 False


In [24]:
for m in g.nodes():
    if origin == m:
        continue
    path = nx.shortest_path(g, origin, m)
    oracle = len(path) - 1
    visited = {}
    test = shortestPath(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem)
    testPath = list(map(int, test[1].split("mem")[1:]))
    print(
        f"{origin}_{m:02} =>",
        oracle,
        test[0],
        "T  " if oracle == test[0] else "F X",
        path,
        testPath,
        nx.is_path(g, testPath),
    )

0_01 => 4 4 T   [0, 18, 6, 21, 1] [24, 20, 5, 1, 9] False
0_02 => 3 3 T   [0, 29, 7, 2] [24, 4, 27, 7] False
0_03 => 4 4 T   [0, 29, 17, 12, 3] [24, 26, 19, 3, 12] False
0_04 => 2 2 T   [0, 27, 4] [24, 4, 27] False
0_05 => 3 3 T   [0, 18, 6, 5] [24, 20, 5, 23] False
0_06 => 2 2 T   [0, 18, 6] [24, 20, 5] False
0_07 => 2 2 T   [0, 29, 7] [24, 26, 2] False
0_08 => 4 4 T   [0, 29, 17, 19, 8] [24, 26, 19, 8, 19] False
0_09 => 3 3 T   [0, 29, 17, 9] [24, 26, 19, 1] False
0_10 => 1 1 T   [0, 10] [24, 14] False
0_11 => 4 4 T   [0, 29, 22, 13, 11] [24, 26, 28, 11, 13] False
0_12 => 3 3 T   [0, 29, 17, 12] [24, 26, 19, 3] False
0_13 => 3 3 T   [0, 29, 22, 13] [24, 26, 28, 11] False
0_14 => 2 2 T   [0, 10, 14] [24, 14, 10] False
0_15 => 3 3 T   [0, 27, 4, 15] [24, 4, 27, 12] False
0_16 => 3 3 T   [0, 18, 20, 16] [24, 20, 16, 20] False
0_17 => 2 2 T   [0, 29, 17] [24, 26, 19] False
0_18 => 1 1 T   [0, 18] [24, 20] False
0_19 => 3 3 T   [0, 29, 17, 19] [24, 26, 19, 8] False
0_20 => 2 2 T   [0, 18,

In [ ]:
testCheckEdge(G, g, THRESHOLD)